# Importação das bibliotecas e Criação dos dataframes

In [3]:
import pandas as pd
import sys
sys.path.append('../')

pd.set_option('display.float_format', lambda x: '%0f' % x)

df_energy = pd.read_csv('../datasets/raw/consumo_energia_eletrica.csv', sep=',')
df_region = pd.read_csv('../datasets/raw/estado_regiao.csv', sep=';')

## Analise inicial dos dados de consumo

In [9]:
df_energy.head()

,ano,mes,sigla_uf,tipo_consumo,numero_consumidores,consumo
0,2004,1,TO,Total,NaN,65876
1,2004,1,BA,Total,NaN,1444451
2,2004,1,PR,Total,NaN,1596274
3,2004,1,RS,Total,NaN,1780912
4,2004,1,GO,Total,NaN,630624


In [7]:
df_energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39897 entries, 0 to 39896
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ano                  39897 non-null  int64  
 1   mes                  39897 non-null  int64  
 2   sigla_uf             39897 non-null  object 
 3   tipo_consumo         39897 non-null  object 
 4   numero_consumidores  26937 non-null  float64
 5   consumo              39897 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 1.8+ MB


A partir da analise inicial dos dados que alguns formatos não correspondem ao que seria esperado para natureza do field. \
Como por exemplo, o field numero_consumidores que é apresentado incialmente como float64. Também foi possível identificar a presença de valores nulos no mesmo field.

In [11]:
df_energy.duplicated().sum()


np.int64(1017)

Foram identificadas 1017 linhas duplicadas a serem removidas do dataframe 

In [14]:
df_energy.drop_duplicates(inplace=True)

In [17]:
df_energy.describe().round(2)

,ano,mes,numero_consumidores,consumo
count,38880.000000,38880.000000,25920.000000,38880.000000
mean,2013.500000,6.500000,682303.380000,611732.010000
std,5.770000,3.450000,1839311.690000,1208066.850000
min,2004.000000,1.000000,52.000000,354.000000
25%,2008.750000,3.750000,18956.500000,85476.000000
50%,2013.500000,6.500000,108174.500000,221062.000000
75%,2018.250000,9.250000,442412.000000,596699.000000
max,2023.000000,12.000000,19201264.000000,12809742.000000


Ações de correção na base: 
 1. Alterar o tipo da coluna número consumidores de float para int
 2. Alterar o tipo da coluna consumo de int para float 
 3. Tratar os dados nulos da coluna número consumidores 
 4. Tratar possíveis outliers da coluna consumo


In [25]:
df_energy_custom = df_energy
df_energy_custom['consumo'] = df_energy_custom['consumo'].astype('float')
df_energy_custom.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38880 entries, 0 to 38879
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ano                  38880 non-null  int64  
 1   mes                  38880 non-null  int64  
 2   sigla_uf             38880 non-null  object 
 3   tipo_consumo         38880 non-null  object 
 4   numero_consumidores  25920 non-null  float64
 5   consumo              38880 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 2.1+ MB


In [53]:

df_energy_custom.groupby(['sigla_uf','tipo_consumo'])['numero_consumidores'].sum().reset_index()

,sigla_uf,tipo_consumo,numero_consumidores
0,AC,Cativo,0.000000
1,AC,Comercial,4376707.000000
2,AC,Industrial,140290.000000
3,AC,Outros,6353093.000000
4,AC,Residencial,42270581.000000
...,...,...,...
157,TO,Comercial,7888898.000000
158,TO,Industrial,480289.000000
159,TO,Outros,14157539.000000
160,TO,Residencial,96462459.000000


In [40]:
df_energy_custom.isnull().sum()

ano                        0
mes                        0
sigla_uf                   0
tipo_consumo               0
numero_consumidores    12960
consumo                    0
dtype: int64

Agrupamos o número de consumidores por tipo de consumo para tentar identificar onde haveria maior concentração de valores nulos.

In [60]:
df_energy_custom.groupby(['tipo_consumo'])['numero_consumidores'].sum().reset_index()

,tipo_consumo,numero_consumidores
0,Cativo,0.000000
1,Comercial,1264142056.000000
2,Industrial,125271021.000000
3,Outros,1110354277.000000
4,Residencial,15185536183.000000
5,Total,0.000000


Foi possível identificar que os tipos de consumo 'Cativo' e 'Total' representam todos os valores nulos identificados

In [61]:
df_energy_custom.groupby(['tipo_consumo'])['consumo'].sum().reset_index()

,tipo_consumo,consumo
0,Cativo,6201517579.000000
1,Comercial,1547394950.000000
2,Industrial,3463509299.000000
3,Outros,1359081971.000000
4,Residencial,2421323423.000000
5,Total,8791313198.000000


A partir dessa constatação levantamos a suspeita de que o tipo de consumo 'total' representava o somatório dos demais tipos não nulos. \
Para testar a hipótese elaboramos os filtros abaixo para comparar o total com o somatório dos tipos 'Comercial', 'Industrial', 'Outros', 'Residencial'.

In [62]:
filter = df_energy_custom['tipo_consumo'].isin(['Comercial', 'Industrial', 'Outros', 'Residencial'])
expenditure_filtered = df_energy_custom[filter]['consumo'].sum()
expenditure_total_type = df_energy_custom[df_energy_custom['tipo_consumo'] == 'Total']['consumo'].sum()


print(f'Total dos tipos sem Cativo e total {expenditure_filtered} e Total Geral {expenditure_total_type}')

Total dos tipos sem Cativo e total 8791309643.0 e Total Geral 8791313198.0


O Teste demonstrou que de fato existe a correlação levantada em hipótese. \
Dessa forma concluímos que o tipo de consumo total exibe dados redundantes e pode ser descartado do dataframe.

Pela mesma lógica, não foi possível identificar nenhuma combinação de somas dos tipos de consumo não nulos que resultasse no valor do tipo 'cativo'. \
Sendo assim é possível assumir que seu valor de consumo não possui relevancia no valor total obtido, e portanto, também pode ser removido do dataframe.

In [63]:
df_energy_custom.dropna(inplace=True)
df_energy_custom.groupby(['tipo_consumo'])['consumo'].sum()

tipo_consumo
Comercial     1547394950.000000
Industrial    3463509299.000000
Outros        1359081971.000000
Residencial   2421323423.000000
Name: consumo, dtype: float64

In [67]:
df_energy_custom['numero_consumidores'] = df_energy_custom['numero_consumidores'].astype('int')
df_energy_custom.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25920 entries, 648 to 38879
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ano                  25920 non-null  int64  
 1   mes                  25920 non-null  int64  
 2   sigla_uf             25920 non-null  object 
 3   tipo_consumo         25920 non-null  object 
 4   numero_consumidores  25920 non-null  int64  
 5   consumo              25920 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 1.4+ MB
